In [1]:
import os
import sys
import math
import heapq
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# 因為D* Lite是反向算法，所以會從終點開始
class DStarLite:    
    def __init__(self, start, end, maze):
        self.start, self.end = start, end
        self.g, self.rhs, self.U = {}, {}, {}  # 節點的優先度
        self.km = 0
        self.graph = maze
        self.x_range, self.y_range = maze.shape
        self.obs = self.buildObs(maze)
        for i in range(self.x_range):
            for j in range(self.y_range):
                self.rhs[(i, j)] = float("inf")
                self.g[(i, j)] = float("inf")

        self.rhs[end] = 0.0
        self.U[end] = self.CalculateKey(end) # 將終點放進去
        self.visited = set() # 有到過
    
    def buildObs(self, maze):
        obs = set()
        for i in range(self.x_range):
            for j in range(self.y_range):
                if maze[i][j] == 1:
                    obs.add((i, j))
        return obs
       
    # 計算節點的估計值
    def CalculateKey(self, current):
       # print("calculateKey", current)
        return [min(self.g[current], self.rhs[current]) + self.H(current) + self.km, min(self.g[current], self.rhs[current])]
   
    # 節點到起點的估計距離
    def H(self, current):
        # manhattan
        # return abs(self.current[0] - self.start[0]) + abs(self.current[1] - self.start[1])
        # euclidean
        return  math.hypot(current[0] - self.start[0], current[1] - self.start[1])   
    
    def ComputePath(self):
        while True:
            s, value = self.FindTopKey() # 檢查當前估計值最小
            # print("Compute", s, value)
            # 檢查是否已走完路徑 or 回到先前路徑
            # 當前估計值有沒有比終點估計值小
            # 終點的實際和估計有沒有相等 一開始都是無窮大所以會相等，等到後面更新就會不一樣了
            if value >= self.CalculateKey(self.start):
                # print("end")
                break
            
            self.U.pop(s) # 將這個點踢出queue
            self.visited.add(s) # 加到路徑

            if value < self.CalculateKey(s): # 更新新的key
                self.U[s] = self.CalculateKey(s)
                #print("newKey", s)
            elif self.g[s] > self.rhs[s]: # 更新實際距離
                #print("newDis", s)
                self.g[s] = self.rhs[s]
                for x in self.get_neighbor(s): # 把鄰居的距離也更新一遍
                    self.UpdateVertex(x)
            else: # 代表這個點不會在最短路徑上面
                #print("not on path")
                self.g[s] = float("inf")
                self.UpdateVertex(s)
                for x in self.get_neighbor(s):
                    self.UpdateVertex(x)
            # print("queue", self.U.keys())
            
    # return the min key's value
    def FindTopKey(self):
        s = min(self.U, key = self.U.get) # key, value
        return s, self.U[s]

    def get_neighbor(self, current):
       # for step in [(0,-50), (0,50), (-50,0), (50,0), (50,50), (-50,50), (50,-50), (-50,-50)]: 
        nei_list = set()
        for step in [(-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1)]:
            neighbor = (current[0] + step[0], current[1] + step[1])
           
            # Make sure within range
            if neighbor[0] < 0 or neighbor[1] < 0 or neighbor[0] >= self.x_range or neighbor[1] >= self.y_range: 
                continue
            # Make sure walkable terrain
            if neighbor in self.obs:
                continue

            nei_list.add(neighbor)
        # print("neighbor",nei_list)
        return nei_list
    
    def UpdateVertex(self, current):
        if current != self.end: # 不是起點
            #print(current, self.get_neighbor(current))
            self.rhs[current] = float("inf")
            for x in self.get_neighbor(current):
                self.rhs[current] = min(self.rhs[current], self.g[x] + self.cost(current, x))
        
        if current in self.U: # 如果自己還在queue裡面要拿掉
            self.U.pop(current)

        if self.g[current] != self.rhs[current]:
            self.U[current] = self.CalculateKey(current)

    def cost(self, current, neighbor):
        # 點有問題
        if self.is_collision(current, neighbor):
            return float("inf")

        return math.hypot(neighbor[0] - current[0], neighbor[1] - current[1])
    
    def is_collision(self, current, neighbor):
        # 點在障礙物上
        if current in self.obs or neighbor in self.obs:
            return True

        if current[0] != neighbor[0] and current[1] != neighbor[1]:
            if neighbor[0] - current[0] == current[1] - neighbor[1]:
                s1 = (min(current[0], neighbor[0]), min(current[1], neighbor[1]))
                s2 = (max(current[0], neighbor[0]), max(current[1], neighbor[1]))
            else:
                s1 = (min(current[0], neighbor[0]), max(current[1], neighbor[1]))
                s2 = (max(current[0], neighbor[0]), min(current[1], neighbor[1]))

            if s1 in self.obs or s2 in self.obs:
                return True

        return False
    def extract_path(self):
        path = [self.start]
        s = self.start

        for k in range(100):
            g_list = {}
            for x in self.get_neighbor(s):
                if not self.is_collision(s, x):
                    g_list[x] = self.g[x]
            s = min(g_list, key=g_list.get)
            path.append(s)
            if s == self.end:
                break

        return list(path)
    
    def changemap(self, maze):
        for x in range(self.x_range):
            for y in range(self.y_range):
                if self.graph[x][y] != maze[x][y]:
                    # 更新 self.graph 中的值
                    self.graph[x][y] = maze[x][y]
                    
                    if (x, y) not in self.obs:
                        self.obs.add((x, y))
                        self.g[(x, y)] = float("inf")
                        self.rhs[(x, y)] = float("inf")
                    else:
                        self.obs.remove((x, y))
                        self.UpdateVertex((x, y))
                        
                    for s in self.get_neighbor((x, y)):
                        self.UpdateVertex(s)

        # 在所有點都找完後，一次性計算路徑
        self.visited = set()
        self.ComputePath()



def main(): 
    start = (0, 0)
    end = (8, 8)

    path = DStarLite(start, end, np.zeros((9, 10), dtype=np.uint16))
    path.ComputePath()
    print(path.extract_path())
    maze = [[0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],]
    path.changemap(maze)
    path.ComputePath()
    print(path.extract_path())

    maze = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],]    
    path.changemap(maze)
    path.ComputePath()
    print(path.extract_path())

    maze = [[0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],]    
    path.changemap(maze)
    path.ComputePath()
    print(path.extract_path())

if __name__ == '__main__':
    main()
    

[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8)]
[(0, 0), (1, 1), (2, 2), (3, 3), (4, 3), (5, 3), (5, 4), (5, 5), (6, 6), (7, 7), (8, 8)]
[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 6), (2, 7), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (8, 8)]
[(0, 0), (1, 1), (2, 2), (3, 3), (4, 3), (5, 3), (6, 3), (7, 3), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7), (8, 8)]
